In [1]:
import json
import pandas as pd
import time

In [2]:
org_dir = 'Lactiplantibacillus_plantarum'

In [3]:
summary = pd.read_csv('../data/'+ org_dir + '/' + 'df_pangene_summary_v2.csv', index_col = 0, low_memory = False)
annotation = pd.read_csv('../data/'+ org_dir + '/' + 'df_pangene_eggnog_summary.csv', index_col = 0, low_memory = False)
COG_category = pd.read_csv("../data/COG_category.csv")
annotation["COG_Categories"] = "["+ annotation["COG_category"] + "]" + annotation["COG_category_name"]
COG_pan_class = pd.merge(summary["pangenome_class_2"], annotation[["COG_Categories","COG_category_name","COG_category"]], on = ["Gene"])

In [19]:
COG_category

,Categories,Function details
0,A,RNA processing and modification
1,B,Chromatin Structure and dynamics
2,C,Energy production and conversion
3,D,Cell cycle control and mitosis
4,E,Amino Acid metabolis and transport
5,F,Nucleotide metabolism and transport
6,G,Carbohydrate metabolism and transport
7,H,Coenzyme metabolis
8,I,Lipid metabolism
9,J,Tranlsation


In [5]:
# initialize the 3 classes list
core = []
accessory = []
rare = []

COG = COG_category.Categories.values.tolist()

for i in COG:
    core.append(len(COG_pan_class.loc[(COG_pan_class["COG_category"].str.contains(i)) & (COG_pan_class["pangenome_class_2"]).str.contains("Core"), :]))
    accessory.append(len(COG_pan_class.loc[(COG_pan_class["COG_category"].str.contains(i)) & (COG_pan_class["pangenome_class_2"]).str.contains("Accessory"), :]))
    rare.append(len(COG_pan_class.loc[(COG_pan_class["COG_category"].str.contains(i)) & (COG_pan_class["pangenome_class_2"]).str.contains("Rare"), :]))

In [36]:
geneclass_df = pd.DataFrame({"Core":core,
             "Accessory":accessory,
             "Rare":rare,
             "Category":"["+ COG_category.iloc[:,0]+"]" + " " + COG_category.iloc[:,1]})
geneclass_df["All"] = geneclass_df.iloc[:,:3].sum(axis=1)
geneclass_df_sorted = geneclass_df.sort_values(by=['All'], ascending=False)

In [37]:
geneclass_df_sorted

,Core,Accessory,Rare,Category,All
23,163,472,6168,[-] Not found in COG,6803
22,223,566,3354,[S] Function Unknown,4143
11,53,168,2930,[L] Replication and repair,3151
10,106,298,1165,[K] Transcription,1569
12,44,202,1317,[M] Cell wall/membrane/envelop biogenesis,1563
6,53,323,1145,[G] Carbohydrate metabolism and transport,1521
4,70,217,709,[E] Amino Acid metabolis and transport,996
15,46,167,647,[P] Inorganic ion transport and metabolism,860
2,31,137,366,[C] Energy production and conversion,534
7,28,109,308,[H] Coenzyme metabolis,445


In [39]:
class_json = {"categories": geneclass_df_sorted["Category"].values.tolist(),
              "Core": geneclass_df_sorted["Core"].values.tolist(),
              "Accessory": geneclass_df_sorted["Accessory"].values.tolist(),
              "Rare" : geneclass_df_sorted["Rare"].values.tolist()}

with open('./json_file/' + org_dir + '_data.json', 'w') as file:
    json.dump(class_json, file)